Check the downloaded model

In [ ]:
import onnx

# Load the model
onnx_model = onnx.load("./data/hair_classifier_v1.onnx")

# Print the names of all output nodes
for output in onnx_model.graph.output:
    print(f"Output name: {output.name}")


Output name: output


Preparation of image


In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from urllib import request
from io import BytesIO

# Function to download image from URL
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

# Function to resize the image to target size (200x200)
def prepare_image(img, target_size=(200, 200)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Preprocessing function (normalization)
def preprocess_image(img):
    # Convert to numpy array and normalize to [0, 1]
    img_np = np.array(img).astype(np.float32) / 255.0  # Ensure the type is float32
    
    # Mean and standard deviation used for image normalization
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    
    # Normalize the image (subtract mean and divide by std)
    img_np = (img_np - mean) / std
    
    # Ensure the type is float32 after all operations
    img_np = img_np.astype(np.float32)
    
    # Reshape for the model input (adding batch dimension)
    img_np = np.transpose(img_np, (2, 0, 1))  # Convert from HWC to CHW format
    img_np = np.expand_dims(img_np, axis=0)  # Add batch dimension: (1, 3, 200, 200)
    
    return img_np

# Function to load the ONNX model
def load_model(model_path):
    session = ort.InferenceSession(model_path)
    return session

# Function to run inference on the image
def run_inference(session, img_np):
    # Get input and output names
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    
    # Ensure the input shape is as expected
    print(f"Input name: {input_name}, Expected input shape: {session.get_inputs()[0].shape}")
    
    # Run inference
    output = session.run([output_name], {input_name: img_np})
    return output

# Main function
def main():
    # Step 1: Download the image from the URL
    image_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
    img = download_image(image_url)
    
    # Step 2: Resize the image to 200x200
    img_resized = prepare_image(img, target_size=(200, 200))
    
    # Step 3: Preprocess the image (normalize and reshape)
    img_np = preprocess_image(img_resized)
    
    # Step 4: Load the ONNX model
    model_path = "./data/hair_classifier_v1.onnx"  # Path to your ONNX model
    session = load_model(model_path)
    
    # Step 5: Run inference on the image
    output = run_inference(session, img_np)
    
    # Step 6: Display the output (probability)
    print(f"Model output (probability): {output[0][0]}")
    
    # Optionally: If the model outputs a binary classification (0 = straight, 1 = curly)
    prediction = "Curly" if output[0][0] > 0.5 else "Straight"
    print(f"Predicted hair type: {prediction}")

# Run the main function
if __name__ == "__main__":
    main()


Input name: input, Expected input shape: ['s77', 3, 200, 200]
Model output (probability): [0.09156627]
Predicted hair type: Straight
